In [ ]:
import pygad
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import pandas as pd
import numpy as np 

from functools import partial
from codes.models.data_form.DataForm import DataTransfo_1SNP, PatientList
import matplotlib.pyplot as plt

In [ ]:
### data constants:
CHR = 1
SNP = 'rs673604'
pheno_method = 'Abby' # Paul, Abby
rollup_depth = 4
Classes_nb = 2 #nb of classes related to an SNP (here 0 or 1)
vocab_size = None # to be defined with data
padding_token = 0
prop_train_test = 0.8
load_data = False
save_data = True
remove_none = True
decorelate = False
equalize_label = False
threshold_corr = 0.9
threshold_rare = 50
remove_rare = 'all' # None, 'all', 'one_class'
compute_features = True
padding = True
list_env_features = []
### data format
batch_size = 20
data_share = 1

In [ ]:
dataT = DataTransfo_1SNP(SNP=SNP,
                         CHR=CHR,
                         method=pheno_method,
                         padding=padding,  
                         pad_token=padding_token, 
                         load_data=load_data, 
                         save_data=save_data, 
                         compute_features=compute_features,
                         prop_train_test=prop_train_test,
                         remove_none=True,
                         equalize_label=equalize_label,
                         rollup_depth=rollup_depth,
                         decorelate=decorelate,
                         threshold_corr=threshold_corr,
                         threshold_rare=threshold_rare,
                         remove_rare=remove_rare, 
                         list_env_features=list_env_features,
                         data_share=data_share)
#patient_list = dataT.get_patientlist()


In [ ]:
data, labels, indices_env, name_envs = dataT.get_tree_data()

In [ ]:
data, labels = DataTransfo_1SNP.equalize_label(data, labels)

In [ ]:
data = data[:, np.var(data, axis=0)0 ]

In [ ]:
np.sum(np.sum(data, axis=0)>30)


In [ ]:
data = data[:,np.sum(data, axis=0)>30]

In [ ]:
nb_phenos, nb_patients = data.shape[1], data.shape[0]

In [ ]:
nb_patterns = 100
num_genes = nb_phenos
num_generations = 20
num_parents_mating = 50
gene_space = [0, 1, 2]
p_ini = 0.005

In [ ]:
def predict_pattern(pattern, pheno_sequence):
    if (np.sum(pheno_sequence[pattern==0]-1) == 0) and (np.sum(pheno_sequence[pattern==1]) ==0):
        return 1
    else:
        return 0
# Function to initialize a random population with valid binary values
def initialize_population(nb_patterns, num_genes, p_ini):
    return np.random.choice([0, 1, 2], p=[0.3, 0.3, 0.4], size=(nb_patterns, num_genes))

def one_point_crossover(parents, offspring_size, ga_instance):
    offspring = []
    idx = 0
    while len(offspring) != offspring_size[0]:
        parent1 = parents[idx % parents.shape[0], :].copy()
        parent2 = parents[(idx + 1) % parents.shape[0], :].copy()

        random_split_point = np.random.choice(range(offspring_size[1]))

        parent1[random_split_point:] = parent2[random_split_point:]

        offspring.append(parent1)

        idx += 1

    return np.array(offspring)

def mutation_func(offspring, ga_instance):

    for chromosome_idx in range(offspring.shape[0]):
        random_gene_idx = np.random.choice(range(offspring.shape[1]))

        offspring[chromosome_idx, random_gene_idx] += np.random.randint(0,3)

    return offspring


def fitness_function(ga_instance, solution, solution_idx):
    predict_pattern = partial(ga_instance.predict_pattern, solution)
    predictions = np.apply_along_axis(predict_pattern, arr=ga_instance.data, axis=1)
    return np.sum(predictions==ga_instance.labels) / len(predictions)



In [ ]:
# Initialize the genetic algorithm
ga_instance = pygad.GA(
    num_generations=num_generations,
    num_parents_mating=nb_patterns,
    sol_per_pop=nb_patterns,
    num_genes=num_genes,
    fitness_func=fitness_function,
    gene_space=gene_space,
    initial_population=initialize_population(nb_patterns, num_genes, p_ini),
    crossover_type=one_point_crossover,
    mutation_type=mutation_func
)

ga_instance.data = data
ga_instance.labels = labels
ga_instance.predict_pattern = predict_pattern

In [ ]:
best_solution, best_fitness, _ = ga_instance.best_solution()

print("Meilleure solution:", best_solution)
print("Meilleur fitness:", best_fitness)

predict_pattern_test = partial(predict_pattern, best_solution)
predictions = np.apply_along_axis(predict_pattern_test, arr=ga_instance.data, axis=1)

np.sum(predictions)

In [ ]:
solution = initialize_population(nb_patterns, num_genes,p_ini)[0]


In [ ]:
pheno_sequence = data[0]
np.sum(pheno_sequence)

In [ ]:
predict_pattern_u = partial(ga_instance.predict_pattern, solution)
predictions = np.apply_along_axis(predict_pattern_u, arr=ga_instance.data, axis=1)
np.sum(predictions==ga_instance.labels)

In [ ]:
predict_pattern_u(data[3])

In [ ]:
np.sum(best_solution)

In [ ]:
import pygad

# Define genetic algorithm parameters
num_generations = 50  # This is num_generations
num_parents_mating = 5
sol_per_pop = 10
def fitness_function(ga_instance, solution, solution_idx):
    # Fitness is the number of ones in the binary string
    return np.sum(solution)
# Initialize the genetic algorithm
ga_instance = pygad.GA(
    num_generations=num_generations,
    num_parents_mating=num_parents_mating,
    sol_per_pop=sol_per_pop,
    num_genes=num_genes,
    fitness_func=fitness_function,
)

# Run the genetic algorithm
ga_instance.run()


In [ ]:
ga_instance